Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Исследование надежности заемщиков.

## Шаг 1. Обзор данных

In [33]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## Шаг 2. Предобработка данных

###  Заполнение пропусков

In [34]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

C помощью isna я на нашёл где имеются пропуски это days_employed & total_income

In [35]:
pass_days= data['days_employed'].isna().sum()
pass_income= data['total_income'].isna().sum()
print(pass_days, pass_income)


2174 2174


In [36]:
days_employed_med = data['days_employed'].median()
total_income_med = data['total_income'].median()
print(days_employed_med)
print(total_income_med)

-1203.369528770489
145017.93753253992


присвоил переменным значения медианных значений столбцов days_employed & total_income соответсвенно

In [37]:
data['days_employed'] = data['days_employed'].fillna(days_employed_med)
data['total_income'] = data['total_income'].fillna(total_income_med)
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

С помощью fillna заполнил медианными значениями пропуски и проверил с помощью isna
ознакомившись с данными, я понял что распределение не совсем нормальное, поэтому и было принято решение использовать медианное значение, тк оно более устойчиво к шуму/выплескам

In [38]:
all_days = data['days_employed'].count()
all_income = data['total_income'].count()

procent_days = pass_days / all_days
procent_income = pass_income / all_income
print('Процентные отношения {:.0%}'.format(procent_days))
print('Процентные отношения {:.0%}'.format(procent_income))


Процентные отношения 10%
Процентные отношения 10%


После заполнения пропусков я посчитал количество значений для столбцов days_employed & total_income и процентное отношение количества пропусков в общему значению в стобцах соответственно 


### Проверка данных на аномалии и исправления.

In [39]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,56557.335698,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,134922.319298,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-385.106616,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


С помощью info и describe ознакомился с данными 

data['days_employed'] = data['days_employed'].abs()
data['children'] = data['children'].abs()
data.info()

С помощью abs() избавился от отрицательных значений

In [40]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

20 и -1 скорее всего опечатки - заменю их на 2 и 1

In [41]:
data.loc[data['children']==20,'children']=2
data.loc[data['children']==-1,'children']=1
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [42]:
(data[pd.isnull(data['days_employed'])==False]['days_employed']//365).sort_values() ###вывел стаж в годах

16335     -51.0
4299      -49.0
7329      -46.0
16825     -45.0
17838     -45.0
          ...  
10006    1100.0
7794     1100.0
5716     1100.0
10484    1100.0
4949     1100.0
Name: days_employed, Length: 21525, dtype: float64

In [43]:
data['education'].value_counts() ### замечены отличия в регистре

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [44]:
data['education_id'].value_counts() ### никаких отклонений не замечено

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [45]:
data['family_status'].value_counts() ### тоже без комментариев

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [46]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [47]:
data['gender'].value_counts() ### xna непонятное значение на свое усмотрение отнесу его или к мужчинам или к женщинам

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [48]:
data.loc[data['gender']=='XNA','gender'] = 'F'
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

In [49]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [50]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [51]:
(data['total_income']//10000).describe()

count    21525.000000
mean        16.016771
std          9.789133
min          2.000000
25%         10.000000
50%         14.000000
75%         19.000000
max        226.000000
Name: total_income, dtype: float64

In [52]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Изменение типов данных.

In [53]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


C помощью astype изменил тип данных

### Удаление дубликатов.

In [54]:
data.duplicated().sum()

54

In [55]:
data['education'].head(20)

0                  высшее
1                 среднее
2                 Среднее
3                 среднее
4                 среднее
5                  высшее
6                  высшее
7                 СРЕДНЕЕ
8                  ВЫСШЕЕ
9                 среднее
10                 высшее
11                среднее
12                среднее
13    неоконченное высшее
14                 высшее
15                среднее
16                среднее
17                 высшее
18                среднее
19                СРЕДНЕЕ
Name: education, dtype: object

In [56]:
print('Дубликатов в столбце:', data['education'].duplicated().sum())
print('Разновидность дубликатов:')
print(data['education'].value_counts())

Дубликатов в столбце: 21510
Разновидность дубликатов:
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In [57]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())
print(data['education'].unique())


среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [58]:
data['education'] = data['education'].drop_duplicates()
print(data['education'].value_counts())

высшее                 1
неоконченное высшее    1
среднее                1
ученая степень         1
начальное              1
Name: education, dtype: int64


Проанализировав полученные значения - пришел к выводу, что причиной появления дубликатов стала чувстительность к регистру, привел все значения к общему виду и командой unique проверил свою теорию.

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

education_voc = data[['education_id','education']]
display(education_voc)
status_voc = data[['family_status_id','family_status']]
display(status_voc)

voc_upd = data.drop(columns = ['education', 'family_status'], axis=1)
display(voc_upd)
voc_upd.info()

In [26]:
education_dict = data.loc[:,['education_id','education']]
data_set = data.drop('education',axis=1)
education_dict = education_dict.drop_duplicates()
education_dict = education_dict.set_index('education_id').T.to_dict('list')
education_dict

{0: ['высшее'],
 1: ['среднее'],
 2: ['неоконченное высшее'],
 3: ['начальное'],
 4: ['ученая степень']}

In [27]:
family_status_dict = data.loc[:,['family_status_id','family_status']]
data_set = data.drop('family_status',axis=1)
family_status_data = family_status_dict.drop_duplicates()
family_status_dict = family_status_dict.set_index('family_status_id').T.to_dict('list')
family_status_dict

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.


{0: ['женат / замужем'],
 1: ['гражданский брак'],
 2: ['вдовец / вдова'],
 3: ['в разводе'],
 4: ['Не женат / не замужем']}

### Категоризация дохода.

In [28]:
total_income_category = []
for item in data['total_income']:
        if item < 30000:
            total_income_category.append('E')
        elif item < 50000:
            total_income_category.append('D')
        elif item < 200000:
            total_income_category.append('C')
        elif item < 1000000:
            total_income_category.append('B')
        else:
            total_income_category.append('A')
            
data['total_income_category'] = total_income_category
display(data[data['total_income_category'] == 'E'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
934,0,348253.361682,69,среднее,1,женат / замужем,0,F,пенсионер,0,29426,недвижимость,E
1184,0,401575.196728,55,среднее,1,гражданский брак,1,M,пенсионер,0,29749,на проведение свадьбы,E
1598,0,359726.104207,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695,на проведение свадьбы,E
4057,0,386389.497407,64,среднее,1,женат / замужем,0,F,пенсионер,1,29154,строительство недвижимости,E
8273,0,-615.774147,46,среднее,1,женат / замужем,0,F,сотрудник,0,26533,покупка коммерческой недвижимости,E
9070,0,340952.765703,60,среднее,1,Не женат / не замужем,4,F,пенсионер,0,24457,операции с коммерческой недвижимостью,E
10068,0,-4959.893007,43,среднее,1,женат / замужем,0,F,сотрудник,0,25227,высшее образование,E
10622,0,385664.024835,67,среднее,1,женат / замужем,0,F,пенсионер,0,27776,заняться образованием,E
10881,0,347356.519176,71,среднее,1,женат / замужем,0,M,пенсионер,0,22472,операции со своей недвижимостью,E
12052,1,354563.609108,55,начальное,3,гражданский брак,1,F,пенсионер,0,25308,дополнительное образование,E


### Категоризация целей кредита.

In [29]:
data_purpose = data['purpose'].unique()
print(data_purpose)

def purpose_category(purpose_mess):
    try:
        if('жиль' in purpose_mess) or ('недвиж' in purpose_mess):
            return 'операции с недвижимостью'
        elif 'авто' in purpose_mess:
            return 'операции с автомобилем'
        elif 'сва' in purpose_mess:
            return 'проведение свадьбы'
        elif 'обр' in purpose_mess:
            return 'получение образования'
        else:
            print('неподходящее значение', purpose_mess)
            return ''
    except:
            print('ошибочное значение', purpose_mess)
            return ''

data['purpose_category'] = data['purpose'].apply(purpose_category)
data.head(10)
        

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [30]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

## Ответы на вопросы.

#### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

pivot_children =  pd.pivot_table(data,index='children',columns='debt',values='dob_years', aggfunc='count')
pivot_children = pivot_children.fillna(0)
display(pivot_children)

In [31]:

data_set_grouped=data.groupby('children').agg({'debt':['count','sum']})

data_set_grouped=data_set_grouped.assign(group_share_in_total = data_set_grouped['debt']['count']/data_set_grouped['debt']['count'].sum(),\
       debt_children_rate=data_set_grouped['debt']['sum']/data_set_grouped['debt']['count'],\
       repaid_rate = 1-data_set_grouped['debt']['sum']/data_set_grouped['debt']['count'])
data_set_grouped

debt       group_share_in_total debt_children_rate repaid_rate
          count   sum                                                    
children                                                                 
0         14149  1063             0.657329           0.075129    0.924871
1          4865   445             0.226016           0.091470    0.908530
2          2131   202             0.099001           0.094791    0.905209
3           330    27             0.015331           0.081818    0.918182
4            41     4             0.001905           0.097561    0.902439
5             9     0             0.000418           0.000000    1.000000

#### Вывод 1: Люди у которых не было задолженности по возврату имели пятерых детей

#### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

pivot_fam = data.pivot_table(index='family_status', columns='debt', values='dob_years', aggfunc='count')
display(pivot_fam)

In [32]:
def status_from_dict(index_id):
    new_index = []
    for value in index_id:
        new_index.append(family_status_dict[value])
    return new_index

family_pivot = data.pivot_table(index='family_status_id',values='debt', aggfunc=['sum','count','mean'])
family_pivot = family_pivot.assign(group_share_in_total = family_pivot['count']/family_pivot['count'].sum(),\
                                  repaid_rate = 1-family_pivot['mean'])

family_pivot['family_status'] = status_from_dict(family_pivot.index)

family_pivot = family_pivot.sort_values(by=('repaid_rate',''),ascending=False)
family_pivot = family_pivot.rename(columns={'debt':'','sum':'unpaid_qunatity','count':'total_quantity','mean':'unpaid_rate'})
family_pivot

,unpaid_qunatity,total_quantity,unpaid_rate,group_share_in_total,repaid_rate,family_status
,,,,,,
family_status_id,,,,,,
2,63,960,0.065625,0.044599,0.934375,[вдовец / вдова]
3,85,1195,0.071130,0.055517,0.928870,[в разводе]
0,931,12380,0.075202,0.575145,0.924798,[женат / замужем]
1,388,4177,0.092890,0.194053,0.907110,[гражданский брак]
4,274,2813,0.097405,0.130685,0.902595,[Не женат / не замужем]


#### Вывод 2: Не заметил зависимости

####  Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

pivot_total = data.pivot_table(index='total_income_category', columns='debt', values='dob_years', aggfunc='count')
display(pivot_total)

In [38]:

income_dict = {'A':'      0 -  30 000',
              'B' :' 30 001 -  50 000',
              'C' :' 50 001 - 200 000',
              'D' :'200 001 -1000 000',
              'E': '1 000 001 ------->'}

def income_from_dict(index_id):
    new_index = []
    for value in index_id:
        new_index.append(income_dict[value])
    return new_index

income_pivot = data.pivot_table(index='total_income_category',values='debt', aggfunc=['sum','count','mean'])
income_pivot = income_pivot.assign(group_share_in_total = income_pivot['count']/income_pivot['count'].sum(),\
                                  repaid_rate = 1-income_pivot['mean'])

income_pivot['family_status'] = income_from_dict(income_pivot.index)

income_pivot = income_pivot.sort_values(by=('repaid_rate',''),ascending=False)

income_pivot = income_pivot.rename(columns={'debt':'','sum':'unpaid_qunatity','count':'total_quantity','mean':'unpaid_rate'})
income_pivot

,unpaid_qunatity,total_quantity,unpaid_rate,group_share_in_total,repaid_rate,family_status
,,,,,,
total_income_category,,,,,,
D,21,350,0.060000,0.016260,0.940000,200 001 -1000 000
B,356,5041,0.070621,0.234193,0.929379,30 001 - 50 000
A,2,25,0.080000,0.001161,0.920000,0 - 30 000
C,1360,16087,0.084540,0.747364,0.915460,50 001 - 200 000
E,2,22,0.090909,0.001022,0.909091,1 000 001 ------->


####  Вывод 3: Не заметил зависимости

#### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

pivot_purp = data.pivot_table(index='purpose_category', columns='debt', values='dob_years', aggfunc='count')
display(pivot_purp)

In [37]:

purpose_pivot = data.pivot_table(index='purpose_category',values='debt', aggfunc=['sum','count','mean'])
purpose_pivot = purpose_pivot.assign(group_share_in_total = purpose_pivot['count']/purpose_pivot['count'].sum(),\
                                  repaid_rate = 1-purpose_pivot['mean'])

purpose_pivot = purpose_pivot.sort_values(by=('repaid_rate',''),ascending=False)

purpose_pivot = purpose_pivot.rename(columns={'debt':'','sum':'unpaid_qunatity','count':'total_quantity','mean':'unpaid_rate'})
purpose_pivot

,unpaid_qunatity,total_quantity,unpaid_rate,group_share_in_total,repaid_rate
,,,,,
purpose_category,,,,,
операции с недвижимостью,782,10840,0.072140,0.503600,0.927860
проведение свадьбы,186,2348,0.079216,0.109082,0.920784
получение образования,370,4022,0.091994,0.186852,0.908006
операции с автомобилем,403,4315,0.093395,0.200465,0.906605


####  Вывод 4: Не заметил зависмости 